# ML Zoomcamp Homework 4: Evaluation Metrics for Classification

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score

## Dataset Preparation

Load the Course Lead Scoring dataset and prepare it for analysis.

Dataset: https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

Target variable: `converted` (binary - whether client signed up)

In [10]:
# Load the dataset
df = pd.read_csv('data/course_lead_scoring.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

Dataset shape: (1462, 9)

Column names:
['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income', 'employment_status', 'location', 'interaction_count', 'lead_score', 'converted']

First few rows:


,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


### Handle Missing Values

- Replace missing categorical values with 'NA'
- Replace missing numerical values with 0.0

In [11]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nData types:")
print(df.dtypes)

Missing values per column:
lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

Data types:
lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object


In [12]:
# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()

# Remove target from numerical columns if present
if 'converted' in numerical_cols:
    numerical_cols.remove('converted')

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

# Fill missing values
for col in categorical_cols:
    df[col] = df[col].fillna('NA')

for col in numerical_cols:
    df[col] = df[col].fillna(0.0)

print(f"\nMissing values after filling:")
print(df.isnull().sum().sum())

Categorical columns: ['lead_source', 'industry', 'employment_status', 'location']
Numerical columns: ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

Missing values after filling:
0


### Split the data into train/validation/test sets (60%/20%/20%)

Use `train_test_split` with `random_state=1`

In [13]:
# First split: 80% train+val, 20% test
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

# Second split: 60% train, 20% val (from the 80%)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

print(f"Train set size: {len(df_train)} ({len(df_train)/len(df)*100:.1f}%)")
print(f"Validation set size: {len(df_val)} ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test set size: {len(df_test)} ({len(df_test)/len(df)*100:.1f}%)")
print(f"\nTarget distribution in train:")
print(df_train['converted'].value_counts(normalize=True))

Train set size: 876 (59.9%)
Validation set size: 293 (20.0%)
Test set size: 293 (20.0%)

Target distribution in train:
converted
1    0.621005
0    0.378995
Name: proportion, dtype: float64


In [14]:
# Separate features and target
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

# For full train (train + val) - needed for Question 5
y_full_train = df_full_train['converted'].values

## Question 1: ROC AUC Feature Importance

For each numerical variable, use it as a score (prediction) and compute the AUC with the target variable.

If AUC < 0.5, invert the variable by putting "-" in front.

Which numerical variable has the highest AUC among:
- `lead_score`
- `number_of_courses_viewed`
- `interaction_count`
- `annual_income`

In [15]:
# Numerical variables to test
numerical_vars = ['lead_score', 'number_of_courses_viewed', 'interaction_count', 'annual_income']

auc_scores = {}

for var in numerical_vars:
    # Use the variable as prediction (score)
    scores = df_train[var].values
    
    # Calculate AUC
    auc = roc_auc_score(y_train, scores)
    
    # If AUC < 0.5, invert the variable
    if auc < 0.5:
        auc = roc_auc_score(y_train, -scores)
        print(f"{var}: AUC = {auc:.3f} (inverted)")
    else:
        print(f"{var}: AUC = {auc:.3f}")
    
    auc_scores[var] = auc

# Find the variable with highest AUC
best_var = max(auc_scores, key=auc_scores.get)
print(f"\n✓ Answer: {best_var} with AUC = {auc_scores[best_var]:.3f}")

lead_score: AUC = 0.614
number_of_courses_viewed: AUC = 0.764
interaction_count: AUC = 0.738
annual_income: AUC = 0.552

✓ Answer: number_of_courses_viewed with AUC = 0.764


## Question 2: Training the Model

Apply one-hot encoding using `DictVectorizer` and train a logistic regression model:

```python
LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
```

What's the AUC of this model on the validation dataset? (round to 3 digits)

Options: 0.32 / 0.52 / 0.72 / 0.92

In [16]:
# Prepare features (exclude target 'converted')
features = [col for col in df_train.columns if col != 'converted']

# Convert to dictionaries for DictVectorizer
train_dicts = df_train[features].to_dict(orient='records')
val_dicts = df_val[features].to_dict(orient='records')

# Apply one-hot encoding
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Number of features after one-hot encoding: {X_train.shape[1]}")

Training set shape: (876, 31)
Validation set shape: (293, 31)
Number of features after one-hot encoding: 31


In [17]:
# Train logistic regression model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

# Get prediction probabilities for validation set
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Calculate AUC
auc_val = roc_auc_score(y_val, y_pred_proba)
print(f"✓ Answer: AUC on validation dataset = {round(auc_val, 3)}")

✓ Answer: AUC on validation dataset = 0.817


## Question 3: Precision and Recall

Compute precision and recall for all thresholds from 0.0 to 1.0 with step 0.01.

At which threshold do precision and recall curves intersect?

Options: 0.145 / 0.345 / 0.545 / 0.745

In [ ]:
# Create thresholds from 0.0 to 1.0 with step 0.01
thresholds = np.arange(0.0, 1.01, 0.01)

precisions = []
recalls = []

for threshold in thresholds:
    # Convert probabilities to binary predictions using threshold
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    # Calculate precision and recall
    # Handle edge cases where there are no positive predictions
    if y_pred.sum() == 0:
        precision = 0
    else:
        precision = precision_score(y_val, y_pred, zero_division=0)
    
    recall = recall_score(y_val, y_pred, zero_division=0)
    
    precisions.append(precision)
    recalls.append(recall)

# Convert to numpy arrays for easier manipulation
precisions = np.array(precisions)
recalls = np.array(recalls)

In [ ]:
# Plot precision and recall curves
plt.figure(figsize=(10, 6))
plt.plot(thresholds, precisions, label='Precision', linewidth=2)
plt.plot(thresholds, recalls, label='Recall', linewidth=2)
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision and Recall vs Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Find the intersection point (where precision and recall are closest)
differences = np.abs(precisions - recalls)
intersection_idx = np.argmin(differences)
intersection_threshold = thresholds[intersection_idx]

print(f"Threshold where precision and recall intersect: {intersection_threshold:.3f}")
print(f"Precision at intersection: {precisions[intersection_idx]:.3f}")
print(f"Recall at intersection: {recalls[intersection_idx]:.3f}")
print(f"Difference: {differences[intersection_idx]:.6f}")
print(f"\n✓ Answer: {intersection_threshold:.3f}")

## Question 4: F1 Score

Precision and recall are conflicting. They are often combined into the F1 score:

$$F_1 = 2 \cdot \frac{P \cdot R}{P + R}$$

Compute F1 for all thresholds from 0.0 to 1.0 with increment 0.01.

At which threshold is F1 maximal?

Options: 0.14 / 0.34 / 0.54 / 0.74

In [ ]:
# Calculate F1 scores for all thresholds
f1_scores = []

for i, threshold in enumerate(thresholds):
    p = precisions[i]
    r = recalls[i]
    
    # Calculate F1 score: F1 = 2 * (P * R) / (P + R)
    if p + r == 0:
        f1 = 0
    else:
        f1 = 2 * (p * r) / (p + r)
    
    f1_scores.append(f1)

f1_scores = np.array(f1_scores)

In [ ]:
# Plot F1 score
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, linewidth=2, color='green')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score vs Threshold')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Find the threshold with maximum F1 score
max_f1_idx = np.argmax(f1_scores)
max_f1_threshold = thresholds[max_f1_idx]
max_f1_score = f1_scores[max_f1_idx]

print(f"Threshold with maximum F1 score: {max_f1_threshold:.2f}")
print(f"Maximum F1 score: {max_f1_score:.3f}")
print(f"Precision at max F1: {precisions[max_f1_idx]:.3f}")
print(f"Recall at max F1: {recalls[max_f1_idx]:.3f}")
print(f"\n✓ Answer: {max_f1_threshold:.2f}")

## Question 5: 5-Fold Cross-Validation

Use the `KFold` class from Scikit-Learn to evaluate the model on 5 different folds:

```python
KFold(n_splits=5, shuffle=True, random_state=1)
```

- Iterate over different folds of `df_full_train`
- Train the model: `LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)`
- Use AUC to evaluate the model on validation

What is the standard deviation of the AUC scores across different folds?

Options: 0.0001 / 0.006 / 0.06 / 0.36

In [ ]:
# Initialize KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

auc_scores_cv = []

# Iterate over folds
for fold, (train_idx, val_idx) in enumerate(kfold.split(df_full_train), 1):
    # Split data
    df_fold_train = df_full_train.iloc[train_idx]
    df_fold_val = df_full_train.iloc[val_idx]
    
    y_fold_train = df_fold_train['converted'].values
    y_fold_val = df_fold_val['converted'].values
    
    # Prepare features
    features_fold = [col for col in df_fold_train.columns if col != 'converted']
    fold_train_dicts = df_fold_train[features_fold].to_dict(orient='records')
    fold_val_dicts = df_fold_val[features_fold].to_dict(orient='records')
    
    # One-hot encoding
    dv_fold = DictVectorizer(sparse=False)
    X_fold_train = dv_fold.fit_transform(fold_train_dicts)
    X_fold_val = dv_fold.transform(fold_val_dicts)
    
    # Train model
    model_fold = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict and calculate AUC
    y_fold_pred_proba = model_fold.predict_proba(X_fold_val)[:, 1]
    auc_fold = roc_auc_score(y_fold_val, y_fold_pred_proba)
    
    auc_scores_cv.append(auc_fold)
    print(f"Fold {fold}: AUC = {auc_fold:.3f}")

# Calculate statistics
mean_auc = np.mean(auc_scores_cv)
std_auc = np.std(auc_scores_cv)

print(f"\nMean AUC: {mean_auc:.3f}")
print(f"Standard deviation of AUC: {std_auc:.3f}")
print(f"\n✓ Answer: {std_auc:.3f}")

## Question 6: Hyperparameter Tuning

Use 5-Fold cross-validation to find the best parameter `C`.

- Test `C` values: `[0.000001, 0.001, 1]`
- Use `KFold(n_splits=5, shuffle=True, random_state=1)`
- Model: `LogisticRegression(solver='liblinear', C=C, max_iter=1000)`
- Compute the mean score and std (round to 3 decimal digits)

Which `C` leads to the best mean score?

If you have ties, select the score with the lowest std. If you still have ties, select the smallest `C`.

Options: 0.000001 / 0.001 / 1

In [ ]:
# C values to test
c_values = [0.000001, 0.001, 1]

# Store results
results = {}

for c in c_values:
    print(f"\nTesting C = {c}")
    print("-" * 40)
    
    auc_scores_c = []
    
    # Initialize KFold with same parameters
    kfold_c = KFold(n_splits=5, shuffle=True, random_state=1)
    
    # Iterate over folds
    for fold, (train_idx, val_idx) in enumerate(kfold_c.split(df_full_train), 1):
        # Split data
        df_fold_train = df_full_train.iloc[train_idx]
        df_fold_val = df_full_train.iloc[val_idx]
        
        y_fold_train = df_fold_train['converted'].values
        y_fold_val = df_fold_val['converted'].values
        
        # Prepare features
        features_fold = [col for col in df_fold_train.columns if col != 'converted']
        fold_train_dicts = df_fold_train[features_fold].to_dict(orient='records')
        fold_val_dicts = df_fold_val[features_fold].to_dict(orient='records')
        
        # One-hot encoding
        dv_fold = DictVectorizer(sparse=False)
        X_fold_train = dv_fold.fit_transform(fold_train_dicts)
        X_fold_val = dv_fold.transform(fold_val_dicts)
        
        # Train model with current C
        model_c = LogisticRegression(solver='liblinear', C=c, max_iter=1000)
        model_c.fit(X_fold_train, y_fold_train)
        
        # Predict and calculate AUC
        y_fold_pred_proba = model_c.predict_proba(X_fold_val)[:, 1]
        auc_fold = roc_auc_score(y_fold_val, y_fold_pred_proba)
        
        auc_scores_c.append(auc_fold)
        print(f"  Fold {fold}: AUC = {auc_fold:.3f}")
    
    # Calculate statistics
    mean_auc_c = np.mean(auc_scores_c)
    std_auc_c = np.std(auc_scores_c)
    
    results[c] = {
        'mean': round(mean_auc_c, 3),
        'std': round(std_auc_c, 3),
        'scores': auc_scores_c
    }
    
    print(f"  Mean AUC: {round(mean_auc_c, 3)}")
    print(f"  Std AUC: {round(std_auc_c, 3)}")

In [ ]:
# Summary of results
print("\n" + "="*50)
print("Summary of Hyperparameter Tuning")
print("="*50)

for c in c_values:
    print(f"C = {c:>10}: mean = {results[c]['mean']:.3f}, std = {results[c]['std']:.3f}")

# Find best C (highest mean, or lowest std if tied, or smallest C if still tied)
max_mean = max(results[c]['mean'] for c in c_values)
best_candidates = [c for c in c_values if results[c]['mean'] == max_mean]

if len(best_candidates) > 1:
    # If tied, select the one with lowest std
    min_std = min(results[c]['std'] for c in best_candidates)
    best_candidates = [c for c in best_candidates if results[c]['std'] == min_std]
    
    if len(best_candidates) > 1:
        # If still tied, select smallest C
        best_c = min(best_candidates)
    else:
        best_c = best_candidates[0]
else:
    best_c = best_candidates[0]

print(f"\n✓ Answer: Best C value = {best_c}")
print(f"  Mean AUC: {results[best_c]['mean']}")
print(f"  Std AUC: {results[best_c]['std']}")